In [6]:
import pandas as pd

## 1. Read files and combine files

In [7]:
files = ['monday.csv','tuesday.csv', 'wednesday.csv', 'thursday.csv', 'friday.csv']

In [8]:
def read_csv_file(filename):
    filepath = f'../data/{filename}'
    table = pd.read_csv(filepath, sep=';', parse_dates=['timestamp'])
    return table

In [9]:
df = pd.DataFrame()
for file in files:
    table = read_csv_file(file)
    if df.empty:
        df = table
    else:
        table['customer_no'] = table['customer_no'] + df['customer_no'].max()
        df = pd.concat([df, table], ignore_index=True)


## 2. Add checkout state to uncheckout customers

### create df_check table with column "is_checkout" 

In [10]:
df_check = df.sort_values(by=['customer_no', 'timestamp'], ignore_index=True)

In [11]:
def if_checkout(x):
    if "checkout" in x.values:
        return True
    else:
        return False

In [12]:
df_check['is_checkout'] = df_check.groupby('customer_no')['location'].transform(if_checkout)

### check last state of customers

In [13]:
df_last_record = df_check.groupby(['customer_no']).last()
df_last_record.tail()

,timestamp,location,is_checkout
customer_no,,,
7441,2019-09-06 21:48:00,dairy,False
7442,2019-09-06 21:50:00,checkout,True
7443,2019-09-06 21:50:00,checkout,True
7444,2019-09-06 21:50:00,drinks,False
7445,2019-09-06 21:50:00,spices,False


In [14]:
df_uncheckout = df_last_record[df_last_record['is_checkout'] == False]
df_uncheckout.head()

,timestamp,location,is_checkout
customer_no,,,
1430,2019-09-02 21:49:00,fruit,False
1433,2019-09-02 21:49:00,fruit,False
1437,2019-09-02 21:47:00,dairy,False
1439,2019-09-02 21:46:00,fruit,False
1440,2019-09-02 21:49:00,spices,False


In [15]:
for index, row in df_uncheckout.iterrows():
    customer_no = index
    timestamp = row['timestamp']+pd.DateOffset(minutes=1)
    location = "checkout"
    # print(index, row['timestamp'], row['timestamp']+pd.DateOffset(minutes=1))
    new_row = pd.Series({'timestamp':timestamp, 'customer_no':customer_no, 'location':location})
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
    

## 3. Add entrance to every customer

In [16]:
df_first_record = df.groupby(['customer_no']).first()
df_first_record.head(2)

,timestamp,location
customer_no,,
1,2019-09-02 07:03:00,dairy
2,2019-09-02 07:03:00,dairy


In [17]:
for index, row in df_first_record.iterrows():
    customer_no = index
    timestamp = row['timestamp']-pd.DateOffset(minutes=1)
    location = "entrance"
    # print(index, row['timestamp'], row['timestamp']+pd.DateOffset(minutes=1))
    new_row = pd.Series({'timestamp':timestamp, 'customer_no':customer_no, 'location':location})
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)

## 4. Fill out missing minutes
### Add spend_time column

In [18]:
def spend_time(x):
    return int((x.max() - x.min()).total_seconds()/60)

In [19]:
df['spend_time'] = df.groupby('customer_no')['timestamp'].transform(spend_time)

In [20]:
customers = df['customer_no'].unique()
customers

array([1, 2, 3, ..., 7443, 7444, 7445], dtype=object)

In [21]:
for cus in customers:
    df_cus = df[df['customer_no'] == cus]
    start_time = df_cus.iloc[0]['timestamp']
    periods = df_cus.iloc[0]['spend_time']
    dti = pd.date_range(start_time, periods=periods, freq="T")
    for dt in dti[1:]:
        row = df_cus[df_cus['timestamp'] == dt]
        if row.empty:
            dt_pre = dt-pd.DateOffset(minutes=1)
            row = df[(df['timestamp'] == dt_pre) & (df['customer_no'] == cus)]
            row['timestamp'] = dt
            df = pd.concat([df, row], ignore_index=True)

/var/folders/7p/7zk_728n7bn9r5738_xf01rc0000gn/T/ipykernel_6080/4222750827.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['timestamp'] = dt


In [22]:
df.sort_values(["timestamp", "customer_no"], inplace=True, ignore_index=True)

## 3. Add new column "location_next" using shift function

In [26]:
df.sort_values(by=['customer_no', 'timestamp'], inplace=True)

In [27]:
df.tail()

,timestamp,customer_no,location,spend_time
61370,2019-09-06 21:50:00,7444,drinks,2
61374,2019-09-06 21:51:00,7444,checkout,2
61362,2019-09-06 21:49:00,7445,entrance,2
61371,2019-09-06 21:50:00,7445,spices,2
61375,2019-09-06 21:51:00,7445,checkout,2


In [28]:
df['location_next'] = df['location'].shift(-1)
df

,timestamp,customer_no,location,spend_time,location_next
0,2019-09-02 07:02:00,1,entrance,3,dairy
2,2019-09-02 07:03:00,1,dairy,3,dairy
10,2019-09-02 07:04:00,1,dairy,3,checkout
18,2019-09-02 07:05:00,1,checkout,3,entrance
1,2019-09-02 07:02:00,2,entrance,4,dairy
...,...,...,...,...,...
61370,2019-09-06 21:50:00,7444,drinks,2,checkout
61374,2019-09-06 21:51:00,7444,checkout,2,entrance
61362,2019-09-06 21:49:00,7445,entrance,2,spices
61371,2019-09-06 21:50:00,7445,spices,2,checkout


## 4. chage location_next to "checkout" if location is "checkout"

In [29]:
for index, row in df.iterrows():
    if row['location'] == 'checkout':
        df.at[index, 'location_next'] = 'checkout'
df.head(20)

,timestamp,customer_no,location,spend_time,location_next
0,2019-09-02 07:02:00,1,entrance,3,dairy
2,2019-09-02 07:03:00,1,dairy,3,dairy
10,2019-09-02 07:04:00,1,dairy,3,checkout
18,2019-09-02 07:05:00,1,checkout,3,checkout
1,2019-09-02 07:02:00,2,entrance,4,dairy
3,2019-09-02 07:03:00,2,dairy,4,dairy
11,2019-09-02 07:04:00,2,dairy,4,dairy
19,2019-09-02 07:05:00,2,dairy,4,checkout
32,2019-09-02 07:06:00,2,checkout,4,checkout
4,2019-09-02 07:03:00,3,entrance,3,dairy


In [30]:
df.sort_values(by=['timestamp', 'customer_no'], inplace=True, ignore_index=True)
df

,timestamp,customer_no,location,spend_time,location_next
0,2019-09-02 07:02:00,1,entrance,3,dairy
1,2019-09-02 07:02:00,2,entrance,4,dairy
2,2019-09-02 07:03:00,1,dairy,3,dairy
3,2019-09-02 07:03:00,2,dairy,4,dairy
4,2019-09-02 07:03:00,3,entrance,3,dairy
...,...,...,...,...,...
61371,2019-09-06 21:50:00,7445,spices,2,checkout
61372,2019-09-06 21:51:00,7431,checkout,17,checkout
61373,2019-09-06 21:51:00,7435,checkout,11,checkout
61374,2019-09-06 21:51:00,7444,checkout,2,checkout


In [31]:
df.to_csv('supermarket_df_final.csv', sep=';',index=False)